# Running Pipelines using MIRI image data from MAST

## Table of Contents:
> * [Detector1 Pipeline](#detector1_pipeline)
> * [Resources and Documentation](#resources)
> * [Download Data from MAST](#download_from_mast)
> * [Run Pipeline with Default Configuration](#pipeline_with_defaults)
> * [About Configuration Files](#pipeline_configs)
> * [Run Pipeline with Configuration Files](#pipeline_with_cfgs)
> * [Run Pipeline with Parameters Set Programmatically](#pipeline_no_configs)
> * [Run Individual Steps with Configuration Files](#steps_with_config_files)
> * [Run Level 1, 2, 3, Pipelines in Succession](#pipelines_level_123)

***
<a id=detector1_pipeline></a>
##  Detector1 Pipeline

Stage 1 consists of detector-level corrections that are performed on a group-by-group basis, followed by ramp fitting. 

More information can be found at: https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_detector1.html#calwebb-detector1

> **Inputs**: The inputs to stage 1 processing will usually be level-1b raw files.

> **Outputs**: The output of stage 1 processing is a countrate image per exposure, or per integration for some modes.
    
### Level 1 pipeline:

**Calwebb Detector1**   (jwst.pipeline, calwebb_detector1, Detector1Pipeline)   (calwebb_detector1.cfg)

### Level 1 pipeline steps:

**Group Scale** (jwst.group_scale, group_scale_step, GroupScaleStep)   (group_scale.cfg)

**DQInit** (jwst.dq_init, dq_init_step, DQInitStep)    (dq_init.cfg)

**Saturation** (jwst.saturation, saturation_step, SaturationStep)    (saturation.cfg)

**IPC**   (jwst.ipc,  ipc_step,  IPCStep)    (ipc.cfg)

**Super Bias**   (jwst.superbias,  superbias_step,  SuperBiasStep)    (superbias.cfg)

**Refpix** (jwst.refpix,  refpix_step,  RefpixStep)    (refpix.cfg)

**RSCD**   (jwst.rscd,  rscd_step,  RSCD_Step)   (rscd.cfd)

**First Frame**  (jwst.firstframe, firstframe_step,  FirstFrameStep)   (firstframe.cfg)

**Last Frame**  (jwst.lastframe,  lastframe_step, LastFrameStep)   (lastframe.cfg)

**Linearity**  (jwst.linearity,  linearity_step, LinearityStep)   (linearity.cfg)

**Dark Current**  (jwst.dark_current,  dark_current_step,  DarkCurrentStep)   (dark_current.cfg)

**Peristence**  (jwst.persistence,  persistence_step, PersistenceStep)   (persistence.cfg)

**Jump**  (jwst.jump, jump_step,  JumpStep)  (jump_step.cfg)

**RampFit**   (jwst.ramp_fit,  ramp_fit_step,  RampFitStep)   (ramp_fit_step.cfg)

**Gain Scale**  (jwst.gain_scale,  gain_scale_step,  GainScaleStep)   (gain_scale_step.cfg)

(for more information on individual steps see: https://jwst-pipeline.readthedocs.io/en/latest/jwst/package_index.html)

***
<a id='resources'></a>
## Resources and Documentation

There are several different places to find information on installing and running the pipeline. This notebook will give a shortened description of the steps pulled from the detailed pipeline information pages, but to find more in-depth instructions use the links below. 

>1. JDox: https://jwst-docs.stsci.edu/display/JDAT/JWST+Data+Reduction+Pipeline
>2. Installation page: http://astroconda.readthedocs.io/en/latest/releases.html#pipeline-install
>3. Detailed pipeline information: https://jwst-pipeline.readthedocs.io/en/latest/jwst/introduction.html
>4. Help Desk (click on Pipeline Support): https://stsci.service-now.com/jwst?id=sc_category
>5. GitHub README installation instructions: https://github.com/spacetelescope/jwst/blob/master/README.md


If this is your first time trying to run the pipeline from a jupyter notebook, you need to install the jupyter notebook in your pipeline environment:
>1. In a new terminal, change the directory to your working directory, terminal command: cd [your working directory]
>2. Terminal command: source activate jwst_dev
(or whatever your environment name for the pipeline is)
>3. Terminal command: conda install jupyter
>4. Terminal command: jupyter notebook

**NOTE:** During your first run CRDS may download and cache reference files in $HOME/crds_cache.  On subsequent runs cached files will be used.

## Imports

In [ ]:
import os

from astropy.io import fits
from urllib.request import urlretrieve
import matplotlib.pyplot as plt
%matplotlib inline

from jwst.pipeline import Detector1Pipeline
from jwst.pipeline import Image2Pipeline
from jwst.pipeline import Image3Pipeline
from jwst import datamodels
from jwst.pipeline.collect_pipeline_cfgs import collect_pipeline_cfgs

### Quick & Dirty Plots

In [ ]:
def plot_image(obj, index=None):
    if isinstance(obj, str):
        data = fits.getdata(obj)
    else:
        data = obj.data
    image = data[index] if index is not None else data
    fig = plt.figure(figsize=(12,12))
    ax = fig.add_subplot(1, 1, 1)
    im = ax.imshow(image, origin='lower') # , norm=norm)
    fig.colorbar(im)
    plt.show()

***
<a id="download_from_mast"></a>
## Download Data from MAST

Test data can be downloaded from MAST using Astroquery

In [ ]:
MAST_URI_PREFIX = "https://pwjwdmsauiweb.stsci.edu/portal/Download/file?uri=mast:JWST/product/" 

def download_mast(data_name,  local_name):
    return urlretrieve(MAST_URI_PREFIX + data_name,  local_name)

In [ ]:
download_mast("jw00608002001_02101_00001_mirimage_uncal.fits", "test.fits")

***
<a id="pipeline_with_defaults"></a>
## Run Pipeline With Default Configuration

Pipelines can be run by using the .call() method on the Pipeline class and passing in a data file.   Running a pipeline generally executes each successive step on the output from the previous step.  The end result is an output data model.

In [ ]:
output_model = Detector1Pipeline.call("test.fits")
output_model

***
<a id='pipeline_configs'></a>
## About Configuration Files

Configuration files are optional inputs for each step of the pipeline, as well as for the pipeline itself. These files list step-specific parameters, and can also be used to control which steps are run as part of the pipeline.

You can get the full compliment of configuration files using the collect_pipeline_cfgs convenience function from the command line:

    $ collect_pipeline_cfgs ./cfgs
    
This will store every .cfg file for all JWST instruments in a directory named "cfgs".   Note that default parameters in the config files are not necessarily optimized for any particular instrument.
    
A Python equivalent can also be imported and called from your notebook,  see below.

Each of these configuration files can be customized to control pipeline behavior. For example, the configuration file for the Level 1 detector pipeline is called calwebb_detector1.cfg and contains a list (not necessarily in order) of the steps run as part of the Level 1 pipeline.

```
name = "Detector1Pipeline"
class = "jwst.pipeline.Detector1Pipeline"
save_calibrated_ramp = False

    [steps]
      [[group_scale]]
        config_file = group_scale.cfg
      [[dq_init]]
        config_file = dq_init.cfg
      [[saturation]]
        config_file = saturation.cfg
      [[ipc]]
        skip = True
      [[superbias]]
        config_file = superbias.cfg
      [[refpix]]
        config_file = refpix.cfg
      [[rscd]]
        config_file = rscd.cfg
      [[firstframe]]
        config_file = firstframe.cfg
      [[lastframe]]
        config_file = lastframe.cfg
      [[linearity]]
        config_file = linearity.cfg
      [[dark_current]]
        config_file = dark_current.cfg
      [[persistence]]
        config_file = persistence.cfg
      [[jump]]
        config_file = jump.cfg
      [[ramp_fit]]
        config_file = ramp_fit.cfg
      [[gain_scale]]
        config_file = gain_scale.cfg
```

In this example, the ipc step will be skipped (skip = True).

Note that calwebb_detector1.cfg lists a configuration file for each pipeline step. You can customize a particular pipeline step by editing the parameters in its configuration file. 


### Config File Setup

To obtain all config files in a "cfgs" directory execute the following cell.

In [ ]:
# ! rm -rf ./cfgs    # uncomment to delete your cfgs
if not os.path.exists("./cfgs"):
    collect_pipeline_cfgs('./cfgs')
! ls cfgs

***
<a id="pipeline_with_cfgs"></a>
## Run Pipeline with Configuration Files

In your cfgs directory, edit the file calwebb_detector1.cfg and change this:

      [[dark_current]]
        config_file = dark_current.cfg
        
to this:

      [[dark_current]]
        config_file = dark_current.cfg
        save_results = True

by executing the following cell (for demo purposes only) and printing the resulting calwebb_detector1_with_save.cfg file:

In [ ]:
# This cell is pretending to be you creating and editing a new .cfg file.   In real life,  you'd edit the file.

! cp cfgs/calwebb_detector1.cfg  cfgs/calwebb_detector1_with_save.cfg
! perl -pi -e's/      \[\[dark_current\]\]\n/      \[\[dark_current\]\]\n        save_results = True\n/g' cfgs/calwebb_detector1_with_save.cfg
! cat cfgs/calwebb_detector1_with_save.cfg

# And the result is...

Normally just use menu File -> New -> Terminal and then edit with emacs or vi.  Alternatively use the programmatic methods demoed later.

In [ ]:
output_model = Detector1Pipeline.call("test.fits", config_file='cfgs/calwebb_detector1_with_save.cfg')
output_model

In [ ]:
!ls test_dark_current.fits   # you should see a file since you set save_results=True

In [ ]:
plot_image(output_model.data)

In [ ]:
plot_image("test_dark_current.fits", index=(0,0))

***
<a id="pipeline_no_configs"></a>
## Run Pipeline with Parameters Set Programmatically

You can also run the pipeline without relying on configuration files by setting parameters programmatically, and relying on the defaults in the pipeline.   This is the pure-Python alternative to editing .cfg files.

In [ ]:
! rm -f test_dark_current.fits   # remove the dark current output,  we'll recreate it another way...

det1p = Detector1Pipeline()
det1p.dark_current.save_results = True
det1p.run("test.fits")

Setting save_results=True results in the output of the test_dark_current.fits file which can be passed into the PersistenceStep when run in isolation, next.

In [ ]:
! ls test_dark_current.fits   # make sure save_results worked.

***
<a id="steps_with_config_files"></a>
## Run Individual Steps with Configuration Files

You can also change parameter values in the .cfg files for individual Steps.

Edit the cfgs/persistence.cfg file and change:

```
   input_trapsfilled = ""
   flag_pers_cutoff = 40.
   save_persistence = False
```

to:

```
   input_trapsfilled = ""
   flag_pers_cutoff = 40.
   save_persistence = True
```

This will cause PersistenceStep to output a third output file with suffix “_output_pers”. 

See https://jwst-pipeline.readthedocs.io/en/latest/jwst/persistence/description.html
for more information on the persistence step.

In [ ]:
# This cell is pretending to be you creating and editing a new .cfg file.    In real life, you'd edit the file.

! cp cfgs/persistence.cfg  cfgs/persistence_with_out_pers_save.cfg
! perl -pi -e's/save_persistence = False/save_persistence = True/g' cfgs/persistence_with_out_pers_save.cfg
! cat cfgs/persistence_with_out_pers_save.cfg

# And the result is...

In [ ]:
from jwst.persistence.persistence_step import PersistenceStep

output_model = PersistenceStep.call("test_dark_current.fits", config_file="cfgs/persistence_with_out_pers_save.cfg")

output_model

In [ ]:
! ls *.fits    # save_persistence may not be working...

***
<a id="pipelines_level_123"></a>
## Run Level 1, 2, 3, Pipelines in Succession

This section runs the test data through all stages of processing,  chaining the output of each processing level to the input of the next.   Output can optionally be saved to a file.   Alternatively,  in-memory pipeline output can
be passed directly to the next level.

Documentation on all JWST CAL pipelines can be found here: https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/main.html#pipelines

### Run Level 1

Stage 1 consists of detector-level corrections that are performed on a group-by-group basis, followed by ramp fitting. The output of stage 1 processing is a countrate image per exposure, or per integration for some modes. 

In [ ]:
output1 = Detector1Pipeline.call("test.fits", config_file="cfgs/calwebb_detector1.cfg")

### Run Level 2

Stage 2 processing consists of additional instrument-level and observing-mode corrections and calibrations to produce fully calibrated exposures. The details differ for imaging and spectroscopic exposures, and there are some corrections that are unique to certain instruments or modes. 

In [ ]:
output2 = Image2Pipeline.call(output1, config_file="cfgs/calwebb_image2.cfg")

### Run Level 3

Stage 3 processing consists of routines that work with multiple exposures and in most cases produce some kind of combined product. There are unique pipeline modules for stage 3 processing of imaging, spectroscopic, coronagraphic, AMI, and TSO observations.

In [ ]:
output3 = Image3Pipeline.call(output2[0], config_file="cfgs/calwebb_image3.cfg")